In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer

# 1. 读取数据
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

# 2. 观察数据
print(f"训练集大小: {len(train_df)}")
print(f"测试集大小: {len(test_df)}")
print("\n前5条数据：")
display(train_df.head())

# 3. 简单的补全策略
# 将缺失值填补为 "None" 是完全没问题的
train_df = train_df.fillna("None")
test_df = test_df.fillna("None")

# 4. 看看正负样本比例
print("\n灾难(1) vs 非灾难(0) 的比例：")
print(train_df['target'].value_counts())

# ==========================================
# 🔥 关键补充：在这里初始化 Tokenizer
# ==========================================
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print("\n✅ 准备工作完成！")
print(f"✅ 数据已加载 (train_df)，分词器已就位 (tokenizer: {model_checkpoint})")
print("👉 现在你可以放心地运行下一个格子 (Dataset类) 的代码了，绝对不会报错！")

训练集大小: 7613
测试集大小: 3263

前5条数据：


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1



灾难(1) vs 非灾难(0) 的比例：
target
0    4342
1    3271
Name: count, dtype: int64


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


✅ 准备工作完成！
✅ 数据已加载 (train_df)，分词器已就位 (tokenizer: distilbert-base-uncased)
👉 现在你可以放心地运行下一个格子 (Dataset类) 的代码了，绝对不会报错！


In [3]:
import torch
from torch.utils.data import Dataset

# 定义我们的数据集类
class DisasterDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128, is_test=False):
        self.texts = df.text.values  # 拿到所有的文本
        self.tokenizer = tokenizer   # 拿到你的翻译官
        self.max_len = max_len       # 设定每句话最长多少个词
        self.is_test = is_test       # 标记是不是测试集
        
        # 如果不是测试集，我们要把标签 (0或1) 也拿出来
        if not is_test:
            self.targets = df.target.values

    # 告诉模型我们总共有多少条数据
    def __len__(self):
        return len(self.texts)

    # 核心：这是模型每次拿数据时会调用的函数
    def __getitem__(self, item):
        text = str(self.texts[item])
        
        # 让翻译官干活：把文本变成数字
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # 加上 [CLS] 和 [SEP]
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',     # 不够长的补 0
            truncation=True,          # 太长的切掉
            return_attention_mask=True,
            return_tensors='pt',      # 返回 PyTorch 的张量
        )

        output = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        
        # 如果有标签，也一起返回
        if not self.is_test:
            output['labels'] = torch.tensor(self.targets[item], dtype=torch.long)
            
        return output

# 把刚才读取的 pandas 表格转换成这个 Dataset 对象
# 注意：train_df 和 val_split 是我们在第一步里切分好的
# 确保你之前的 tokenizer 变量还在内存里
full_train_dataset = DisasterDataset(train_df, tokenizer) # 全量数据（可选）
train_dataset = DisasterDataset(train_df, tokenizer)      # 这里简单起见直接用 train_df，进阶可以用切分后的 train_split
test_dataset = DisasterDataset(test_df, tokenizer, is_test=True)

print("✅ 数据集组装完成！")

✅ 数据集组装完成！


In [4]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

# 1. 加载预训练模型
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 2. 定义“评分标准”
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# 3. 设定训练参数 (Training Arguments)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    
    # 🔥🔥🔥 注意这里：我改成了最新的 eval_strategy 🔥🔥🔥
    eval_strategy="epoch",  
    
    save_strategy="epoch",
    report_to="none"
)

# 4. 初始化训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 5. 开始训练！
print("🔥 模型开始训练...")
trainer.train()

2026-01-08 11:26:05.991646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767871566.194291      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767871566.252115      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767871566.740890      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767871566.740932      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767871566.740935      55 computation_placer.cc:177] computation placer alr

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_55/880408638.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🔥 模型开始训练...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.327762,0.875608,0.874188
2,No log,0.291429,0.890976,0.890083


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=476, training_loss=0.3860714054909073, metrics={'train_runtime': 151.2903, 'train_samples_per_second': 100.641, 'train_steps_per_second': 3.146, 'total_flos': 504237152984064.0, 'train_loss': 0.3860714054909073, 'epoch': 2.0})

In [5]:
import pandas as pd

print("🚀 正在让模型预测测试集...")

# 1. 预测 (使用你之前定义好的 trainer 和 test_dataset)
predictions = trainer.predict(test_dataset)

# 2. 转换结果 (把概率变成 0 或 1)
preds = predictions.predictions.argmax(-1)

# 3. 制作表格
submission = pd.DataFrame()
submission['id'] = test_df['id']  # 确保 test_df 还在内存里
submission['target'] = preds

# 4. 保存文件
# 注意：这行代码会把文件保存在最外层，方便你找到
submission.to_csv('submission.csv', index=False)

print("🎉 submission.csv 生成成功！")
print("👇 请看右侧 Output 区域，点击刷新图标 (🔄) 下载！")

🚀 正在让模型预测测试集...


🎉 submission.csv 生成成功！
👇 请看右侧 Output 区域，点击刷新图标 (🔄) 下载！
